# Import libraries and setup

In [1]:
import sys, os, csv, importlib

In [2]:
import numpy as np
import scipy as sc
import scipy.linalg as spl
import scipy.stats as ss
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Import our custom optimization utils
sys.path.insert(0, '../python/')
import grad_utils as model
import cv_utils
import opt_utils

# Key utilities to import ELO + Run BT model

In [4]:
def max_change(beta):
    '''
    get the maximal change in rank in neighboring timepoint based on beta
    '''
    T,N = beta.shape
    arg = np.array([ss.rankdata(-beta[ii]) for ii in range(T)])
    return np.max(abs(arg[1:] - arg[:-1]))

def newton_l2_sq(data,l_penalty):
    '''
    Newton
    '''
    # intiialize optimization
    max_iter = 1000
    ths = 1e-12

    # vanilla PGD parameters
    step_size = 0.03

    # backtracking parameters
    step_init = 0.1
    max_back = 100
    a = 0.2
    b = 0.5
    
    T, N = data.shape[0:2]
    beta = np.zeros(data.shape[:2]).reshape((N * T,1))
    step_size = 1

    # initialize record
    objective_nt = [objective_l2_sq(beta, data, l_penalty)]

    # iteration
    for i in range(max_iter):
        # compute gradient
        gradient = grad_l2_sq(beta, data, l_penalty)[1:]
        hessian = hess_l2_sq(beta, data, l_penalty)[1:,1:]
        # newton update
        beta[1:] = beta[1:] - step_size * sc.linalg.solve(hessian, gradient)

        # objective value
        objective_nt.append(objective_l2_sq(beta, data, l_penalty))

        if objective_nt[-2] - objective_nt[-1] < ths:
            break

    if i >= max_iter:
        print("Not converged.")
        
    return beta.reshape((T,N))

def plot_nfl_round(beta, team_id,season):
    T, N = beta.shape
    year = range(1,17)
    f = plt.figure(1, figsize = (6,4))

    for i in range(N):
        plt.plot(year,beta[:,i], label=team_id['name'][i], color = np.random.rand(3,))
    plt.xlabel("round")
    plt.ylabel("latent parameter")
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1, 1, 0),prop={'size': 5})
    plt.ticklabel_format(style='plain',axis='x',useOffset=False)

    f.savefig("nfl_round_"+str(season)+".pdf", bbox_inches='tight')
        

def get_single_round_pwise(rnd_num, nfl_data_dir, season):
    """
    Gets the pairwise numpy array of score diffences across teams for a single
       round in a season
    """
    fname = "round" + "_" + str(rnd_num).zfill(2) + ".csv"
    fpath = os.path.join(nfl_data_dir, str(season), fname)
    rnd_df = pd.read_csv(fpath)
    pwise_diff = rnd_df.pivot(index='team', columns='team_other',values='diff').values
    pwise_diff[pwise_diff >= 0] = 1
    pwise_diff[pwise_diff < 0] = 0
    pwise_diff[np.isnan(pwise_diff)] = 0
    return pwise_diff

def get_final_rank_season(data_dir, season, team_id, all_rnds, plot = True, 
                          loocv= True, threshold = 3):
    game_matrix_list = np.array([get_single_round_pwise(rnd_num=rnd, nfl_data_dir=data_dir, season=season) 
                                  for rnd in ALL_RNDS])
    
    if loocv:
        lambdas_smooth = np.linspace(0, 5, 21)[1:]
        lambda_cv, nll_cv, beta = cv_utils.loocv(game_matrix_list, lambdas_smooth, 
                                                 opt_utils.newton_l2_sq, 
                                                 verbose='cv', out='terminal')
    
    else:
        lam_list = np.arange(1,80) * 0.5
        val_list = []

        data = game_matrix_list
        for i in range(len(lam_list)):
            val_list.append(max_change(beta = newton_l2_sq(data,lam_list[i])))

        # plt.plot(lam_list,val_list)

        while val_list[-1] > threshold:
            threshold += 1

        ix = next(idx for idx, value in enumerate(val_list) if value <= threshold)
        lambda_star = lam_list[ix]

        beta = newton_l2_sq(data,lambda_star)

    if plot:
        plot_nfl_round(beta = beta,team_id = team_id,season = SEASON)

    arg = np.argsort(-beta,axis=1)
    rank_list = pd.DataFrame(data={(i):team_id['name'][arg[i-1,]].values for i in range(1,17)})
    rank_last = rank_list[16]
    rank_last = pd.DataFrame({'rank':range(len(rank_last))},index = rank_last.values)
    
    return rank_last.sort_index() + 1

def get_elo_rank_season(elo_all, season):
    elo_season = elo_all.iloc[np.where(elo_all['season'] == season)]
    elo_season = elo_season[pd.isnull(elo_season['playoff'])]
    a = elo_season[['team1','elo1_post']]
    a.columns = ['team','elo']
    a = a.reset_index()
    b = elo_season[['team2','elo2_post']]
    b.columns = ['team','elo']
    b = b.reset_index()

    c = pd.concat([a,b])
    c = c.sort_values(by = ['index'])    
    d = c.groupby(by = ['team']).last()
    
    x = d.index.values
    x[np.where(x == 'LAR')] = 'STL'
    x[np.where(x == 'LAC')] = 'SD'
    x[np.where(x == 'JAX')] = 'JAC'
    x[np.where(x == 'WSH')] = 'WAS'
    
    elo_rank = pd.DataFrame({'rank': ss.rankdata(-d['elo'])},index = x).sort_index()
    
    return elo_rank

# Time unit: round

### B-T model estimation

#### Define global variables

In [5]:
NFL_DATA_DIR = "../data/nfl"
SEASONS = range(2009,2016)
ALL_RNDS = range(1, 17)

#### Read in NFL team data

In [6]:
team_id = pd.read_csv(os.path.join(NFL_DATA_DIR, "nfl_id.csv"))
team_id.shape

(32, 2)

#### Run Bradley-Terry Model

In [7]:
bt_out_seasons = [get_final_rank_season(data_dir=NFL_DATA_DIR, 
                                        season=season, 
                                        team_id=team_id, 
                                        all_rnds=ALL_RNDS,
                                        plot=False) for season in SEASONS]

/home/hbong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:76: RuntimeWarning: invalid value encountered in greater_equal
/home/hbong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:77: RuntimeWarning: invalid value encountered in less
/home/hbong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:76: RuntimeWarning: invalid value encountered in greater_equal
/home/hbong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:77: RuntimeWarning: invalid value encountered in less
/home/hbong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:76: RuntimeWarning: invalid value encountered in greater_equal
/home/hbong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:77: RuntimeWarning: invalid value encountered in less
/home/hbong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:76: RuntimeWarning: invalid value encountered in greater_equal
/home/hbong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:77: Runtim

### ELO Estimation

In [8]:
# Get the ELO predictions data from fivethirtyeight
# Source: https://github.com/fivethirtyeight/data/blob/master/nfl-elo/README.md?fbclid=IwAR0vJvH9pf4oYfAqzGlgN6e6RquNJq2rf7ZmzomQHn4p7BYXuwaN3vtsSLA
elo_ft_data = pd.read_csv(os.path.join(NFL_DATA_DIR, "nfl_elo.csv"), na_values='')

# Get all ELO data for the specified seasons
elo_out_seasons = [get_elo_rank_season(elo_all = elo_ft_data, season = season) for season in SEASONS]

## Get ELO vs. BT top 10 table summary

In [9]:
def get_team_season(season_idx, model_season_list):
    model_season = model_season_list[season_idx]
    model_season['rank'] = model_season['rank'].astype(np.int64)
    model_season['team'] = model_season.index
    model_season = model_season.sort_values(by='rank', ascending=True)
    return model_season

def get_join_elo_bt_season(season_num, elo_team_season, bt_team_season, top_n):
    top_season = elo_team_season.merge(bt_team_season, how='left', on=['rank'])
    top_season.columns = ["rank", f"ELO {season_num}", f"BT {season_num}"]
    top_season = top_season[[f"ELO {season_num}", f"BT {season_num}"]].head(top_n)
    #top_season = top_season.reset_index(drop=True)
    return top_season

### Generate table summary

In [10]:
# We want the top 10 teams to be compared
TOP_N = 10

In [11]:
num_seasons = len(elo_out_seasons)
get_elo_seasons = [get_team_season(season_idx=season_idx, model_season_list=elo_out_seasons) for 
                   season_idx in range(num_seasons)]
get_bt_seasons = [get_team_season(season_idx=season_idx, model_season_list=bt_out_seasons) for 
                  season_idx in range(num_seasons)]

elo_bt_join = []
for idx, season in enumerate(SEASONS):
    elo_bt_join.append(get_join_elo_bt_season(season_num=season, 
                                              elo_team_season=get_elo_seasons[idx], 
                                              bt_team_season=get_bt_seasons[idx], 
                                              top_n=TOP_N))
    
# Create a separate rank column    
rank_col = pd.DataFrame(list(range(1, TOP_N + 1)))
rank_col.columns = ['rank']

In [12]:
out_elo_bt = pd.concat(elo_bt_join[-5:], sort=False, axis=1)
out_elo_bt = pd.concat([rank_col, out_elo_bt], sort=False, axis=1)
out_elo_bt

,rank,ELO 2011,BT 2011,ELO 2012,BT 2012,ELO 2013,BT 2013,ELO 2014,BT 2014,ELO 2015,BT 2015
0,1,GB,GB,NE,DEN,SEA,SF,SEA,SEA,SEA,CAR
1,2,NE,NO,DEN,NE,SF,CAR,NE,DEN,CAR,ARI
2,3,NO,NE,GB,SEA,NE,SEA,DEN,GB,ARI,KC
3,4,PIT,SF,SF,MIN,DEN,ARI,GB,NE,KC,SEA
4,5,BAL,PIT,ATL,SF,CAR,NE,DAL,DAL,DEN,MIN
5,6,SF,BAL,SEA,GB,CIN,DEN,PIT,PIT,NE,DEN
6,7,ATL,DET,NYG,IND,NO,NO,BAL,IND,PIT,CIN
7,8,PHI,ATL,CIN,HOU,ARI,CIN,IND,ARI,CIN,PIT
8,9,SD,PHI,BAL,WAS,IND,IND,ARI,BUF,GB,GB
9,10,HOU,SD,HOU,CHI,SD,SD,CIN,DET,MIN,DET


In [13]:
print(out_elo_bt.to_latex(index_names=False, escape=False, index=False, 
                          column_format='c|c|c|c|c|c|c|c|c|c|c|c|c|c|', 
                          header=True, bold_rows=True))

\begin{tabular}{c|c|c|c|c|c|c|c|c|c|c|c|c|c|}
\toprule
 rank & ELO 2011 & BT 2011 & ELO 2012 & BT 2012 & ELO 2013 & BT 2013 & ELO 2014 & BT 2014 & ELO 2015 & BT 2015 \\
\midrule
    1 &       GB &      GB &       NE &     DEN &      SEA &      SF &      SEA &     SEA &      SEA &     CAR \\
    2 &       NE &      NO &      DEN &      NE &       SF &     CAR &       NE &     DEN &      CAR &     ARI \\
    3 &       NO &      NE &       GB &     SEA &       NE &     SEA &      DEN &      GB &      ARI &      KC \\
    4 &      PIT &      SF &       SF &     MIN &      DEN &     ARI &       GB &      NE &       KC &     SEA \\
    5 &      BAL &     PIT &      ATL &      SF &      CAR &      NE &      DAL &     DAL &      DEN &     MIN \\
    6 &       SF &     BAL &      SEA &      GB &      CIN &     DEN &      PIT &     PIT &       NE &     DEN \\
    7 &      ATL &     DET &      NYG &     IND &       NO &      NO &      BAL &     IND &      PIT &     CIN \\
    8 &      PHI &     A

## Calculate average differences for each season from ELO to BTL

In [14]:
av_dif = []
for i in range(7):
    a = elo_out_seasons[i]
    b = bt_out_seasons[i].sort_index()
    av_dif.append(np.mean(abs(a-b)))

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [ ]:
av_dif